- Dates
- Periods
- Calendar
- Schedule
- Interest Rates

In [55]:
import QuantLib as ql
import pandas as pd

# Dates and Periods

In [56]:
day, month, year = 1, 1, 2020
date = ql.Date(day, month, year)
date

Date(1,1,2020)

In [57]:
[x for x in dir(date) if not x.startswith('__')]

['ISO',
 '_old___add__',
 '_old___sub__',
 'dayOfMonth',
 'dayOfYear',
 'endOfMonth',
 'fractionOfDay',
 'fractionOfSecond',
 'from_date',
 'hours',
 'isEndOfMonth',
 'isLeap',
 'localDateTime',
 'maxDate',
 'microseconds',
 'milliseconds',
 'minDate',
 'minutes',
 'month',
 'nextWeekday',
 'nthWeekday',
 'seconds',
 'serialNumber',
 'this',
 'thisown',
 'to_date',
 'todaysDate',
 'universalDateTime',
 'weekday',
 'weekdayNumber',
 'year']

In [58]:
date + 1

Date(2,1,2020)

In [59]:
number_of_periods = 4
period = ql.Months
date + ql.Period(number_of_periods, period)

Date(1,5,2020)

One can also do logical operations using the `Date` object.

In [60]:
date == ql.Date(31, 3, 2015), date > ql.Date(30, 3, 2015), date < ql.Date(1, 4, 2015), date != ql.Date(1, 4, 2015)

(False, True, False, True)

# Calendar

In [61]:
us_calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
pl_calendar = ql.Poland()

In [62]:
period_to_advance = ql.Period(number_of_periods, period)
us_calendar.advance(date, period_to_advance), pl_calendar.advance(date, period_to_advance)

(Date(1,5,2020), Date(4,5,2020))

In [63]:
us_calendar.businessDaysBetween(date, us_calendar.advance(date, period_to_advance))

83

In [64]:
pl_calendar.businessDaysBetween(date, pl_calendar.advance(date, period_to_advance))

84

## join calendars

In [65]:
joint_calendar = ql.JointCalendar(us_calendar, pl_calendar)
joint_calendar.businessDaysBetween(date, joint_calendar.advance(date, period_to_advance))

81

# Schedule

The `Schedule` object is necessary in creating coupon schedules or call schedules. `Schedule` object constructors have the following signature:

```
Schedule(const Date& effectiveDate,
         const Date& terminationDate,
         const Period& tenor,
         const Calendar& calendar,
         BusinessDayConvention convention,
         BusinessDayConvention terminationDateConvention,
         DateGeneration::Rule rule,
         bool endOfMonth,
         const Date& firstDate = Date(),
         const Date& nextToLastDate = Date())  
```

Schedule of:
- 1Y
- monthly (first business day of the month)
- US Bonds

`Schedule` object that will contain dates between `effective_date` and `termination_date` with the `tenor` specifying the `Period` to be `Monthly`. The `calendar` object is used for determining holidays. Here we have chosen the convention to be the day `following` holidays. That is why we see that holidays are excluded in the list of dates.

In [67]:
effective_date = ql.Date(1, 1, 2020)
termination_date = ql.Date(1, 1, 2021)

tenor = ql.Period(ql.Monthly)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)

business_convention = ql.Following
termination_business_convention = ql.Following

date_generation = ql.DateGeneration.Forward
end_of_month = False

schedule = ql.Schedule(
    effective_date,
    termination_date,
    tenor,
    calendar,
    business_convention,
    termination_business_convention,
    date_generation,
    end_of_month
)

pd.DataFrame({'date': list(schedule)})

,date
0,"January 2nd, 2020"
1,"February 3rd, 2020"
2,"March 2nd, 2020"
3,"April 1st, 2020"
4,"May 1st, 2020"
5,"June 1st, 2020"
6,"July 1st, 2020"
7,"August 3rd, 2020"
8,"September 1st, 2020"
9,"October 1st, 2020"


The `Schedule` class can handle generation of dates with irregularity in schedule. The two extra parameters `firstDate` and `nextToLastDate` parameters along with a combination of forward or backward date generation rule can be used to generate short or long stub payments at the front or back end of the schedule. For example, the following combination of `firstDate` and `backward` generation rule creates a short stub in the front on the January 15, 2015.

In [69]:
effective_date = ql.Date(1, 1, 2020)
termination_date = ql.Date(1, 1, 2021)
first_date = ql.Date(15, 1, 2020)
schedule = ql.Schedule(
    effective_date,
    termination_date,
    tenor,
    calendar,
    business_convention,
    termination_business_convention,
    ql.DateGeneration.Backward,
    end_of_month,
    first_date
)

pd.DataFrame({'date': list(schedule)})

,date
0,"January 2nd, 2020"
1,"January 15th, 2020"
2,"February 3rd, 2020"
3,"March 2nd, 2020"
4,"April 1st, 2020"
5,"May 1st, 2020"
6,"June 1st, 2020"
7,"July 1st, 2020"
8,"August 3rd, 2020"
9,"September 1st, 2020"


Using the `backward` generation rule along with the `firstDate` allows us to create a long stub in the front. Below the first two dates are longer in duration than the rest of the dates.

In [71]:
first_date = ql.Date(1, 2, 2015)
effective_date = ql.Date(15, 12, 2014)
termination_date = ql.Date(1, 1, 2016)
schedule = ql.Schedule(
    effective_date,
    termination_date,
    tenor,
    calendar,
    business_convention,
    termination_business_convention,
    ql.DateGeneration.Backward,
    end_of_month, 
    first_date
)

pd.DataFrame({'date': list(schedule)})

,date
0,"December 15th, 2014"
1,"February 2nd, 2015"
2,"March 2nd, 2015"
3,"April 1st, 2015"
4,"May 1st, 2015"
5,"June 1st, 2015"
6,"July 1st, 2015"
7,"August 3rd, 2015"
8,"September 1st, 2015"
9,"October 1st, 2015"


Below the Schedule is generated from a list of dates.

In [72]:
dates = [
    ql.Date(2,1,2015), ql.Date(2, 2,2015),
    ql.Date(2,3,2015), ql.Date(1,4,2015),
    ql.Date(1,5,2015), ql.Date(1,6,2015),
    ql.Date(1,7,2015), ql.Date(3,8,2015),
    ql.Date(1,9,2015), ql.Date(1,10,2015),
    ql.Date(2,11,2015), ql.Date(1,12,2015),
    ql.Date(4,1,2016)
]
rolling_convention = ql.Following

schedule = ql.Schedule(
    dates, calendar,
    rolling_convention
)

pd.DataFrame({'date': list(schedule)})

,date
0,"January 2nd, 2015"
1,"February 2nd, 2015"
2,"March 2nd, 2015"
3,"April 1st, 2015"
4,"May 1st, 2015"
5,"June 1st, 2015"
6,"July 1st, 2015"
7,"August 3rd, 2015"
8,"September 1st, 2015"
9,"October 1st, 2015"


# Interest Rate

In [81]:
annual_interest_rate = 0.02
day_count_convention = ql.ActualActual(ql.ActualActual.ISDA)
compound_type = ql.Compounded
frequency = ql.Annual

interest_rate = ql.InterestRate(
    annual_interest_rate, 
    day_count_convention, 
    compound_type, 
    frequency
)
print(interest_rate)

2.000000 % Actual/Actual (ISDA) Annual compounding


In [83]:
years_passed = 2
interest_rate.compoundFactor(years_passed), interest_rate.discountFactor(years_passed)

(1.0404, 0.9611687812379854)

equivalent rate compounded quarterly

In [86]:
new_ir = interest_rate.equivalentRate(compound_type, ql.Quarterly, years_passed)
print(new_ir)

1.985173 % Actual/Actual (ISDA) Quarterly compounding
